In [1]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [8]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [21]:
data_test = pd.read_csv('data/extra_test.csv', index_col=0)
TARGET = 'net_target'
y = data_test[TARGET].values

In [22]:
y

array([0.40918536, 0.40918536, 0.40918536, ..., 0.49945068, 0.49982259,
       0.46495915])

In [36]:
# Define the log-likelihood function for the AR(p) process
def log_likelihood(params, y, p):
    sigma, alpha = params[0], params[1:]
    T = y.shape[0]
    var_vec = np.zeros(24)
    cov_mat = np.zeros((24, 24))
    
    # Estimate parameters using MLE
    for t in range(p, T):
        var_vec[t%24] += (sigma**2 + np.dot(alpha**2, y[t-p:t]**2))
        for i in range(24):
            for j in range(i, 24):
                cov_mat[i, j] += alpha[i%p]*alpha[j%p]*y[t-i]*y[t-j]
    
    var_mat = np.diag(var_vec)
    cov_mat = cov_mat + cov_mat.T - np.diag(np.diag(cov_mat))
    
    # Calculate the log-likelihood
    log_likelihood = -np.log(np.linalg.det(cov_mat)) - np.dot(np.dot(y[p:].T, np.linalg.inv(cov_mat)), y[p:]) - (T-p)*np.log(2*np.pi)
    
    # Return the negative log-likelihood value
    return -log_likelihood


In [42]:
# Define the order of the AR(p) process
p = 2

# Define the initial values for the parameters
initial_params = [1, 0.5, 0.5]

# Define the bounds for the parameters
bounds = [(1e-5, None), (-1, 1), (-1, 1)]

# Perform the optimization
res = minimize(log_likelihood, initial_params, args=(y, p), bounds=bounds, method='L-BFGS-B')

ValueError: shapes (8710,) and (24,24) not aligned: 8710 (dim 0) != 24 (dim 0)

In [33]:
# check if there is nan in y
np.isnan(y).any()

False